In [ ]:
# test Dealer.shuffle and cards2str functions

import numpy as np
from rlcard.games.tractor.dealer import TractorDealer

np_random = np.random.RandomState()
dealer = TractorDealer(np_random)
dealer.shuffle()
print(dealer.deck)

In [ ]:
# test Judger.playable_cards_from_hand function

from rlcard.games.tractor.judger import TractorJudger

print(TractorJudger.playable_cards_from_hand(['AD','9H','KC','KC','AC','AC','3D','3D','7H','BJ','BJ','RJ','RJ','AD','2D','2D','2H','2H','2S','2S','AS','AS','AD','AD']))

In [ ]:
# test Player.available_actions function

from rlcard.games.tractor import Game
import numpy as np

game = Game()
game.init_game()

print(game.players[0].current_hand)
playable_cards = game.judger.get_playable_cards(game.players[0])
print(playable_cards)
played_cards = max(playable_cards, key=len)

game.players[0].played_cards = played_cards
#game.players[0].played_cards = cards2str([game.players[0].current_hand[10]])
print(game.players[0].played_cards)
print(game.players[1].current_hand)
available_actions = game.players[1].available_actions(game.players[0], game.judger, game.round)
print(available_actions)


In [ ]:
''' Test for round.calc_score_in_round
'''
import numpy as np
from rlcard.games.tractor import Round


np_random = np.random.RandomState()
round = Round(np_random)

round.current_round = [['5S','TS'], ['5H','5S'], ['3S'], ['KH']]
print(round.calc_score_in_round())


In [ ]:
''' generate action space
'''
import numpy as np
from rlcard.games.tractor import Dealer, Judger

np_random = np.random.RandomState()
dealer = Dealer(np)

cards = dealer.deck
# print(cards)

actions = Judger.playable_cards_from_hand(cards)
actions.sort()

actions.extend(['pass', 'pass_score'])
print(actions)
# print(len(actions))

action_index = {','.join(actions[x]):x for x in range(len(actions))}
print(action_index)
print(len(action_index.keys()))



In [ ]:
''' Test Game class
'''
from rlcard.games.tractor import Game

game = Game()
state, player_id = game.init_game()

print(state, "\r\n")

while not game.is_over():
     state, next_id = game.step(state['actions'][0])
     print(state, "\r\n")

print(state)



In [ ]:
''' Test Game class perf
'''
from tqdm import tqdm
from rlcard.games.tractor import Game
import cProfile

def run():
    for iter in tqdm(range(1000)):
        state, player_id = game.init_game()
        while not game.is_over():
            state, next_id = game.step(state['actions'][0])

game = Game()
cProfile.run('run()')





In [1]:
import rlcard
from rlcard.utils import set_global_seed
from rlcard import models

# Make environment
env = rlcard.make('tractor', config={'seed': 0})

# Set a global seed
set_global_seed(0)

# Here we directly load NFSP models from /models module
nfsp_agents = models.load('tractor_nfsp_v1').agents







Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.





Instructions for updating:
Please switch to tf.train.get_global_step

INFO:tensorflow:Restoring parameters from d:\Projects\rlcard\rlcard\models\tractor\20210507_2_nfsp_2_rule_game_reward_only\model


In [35]:
from rlcard.agents import RandomAgent, TractorRuleAgent

# Evaluate the performance. Play with random agents.
random_agent = RandomAgent(env.action_num)
rule_agent = TractorRuleAgent(env.action_num)

# env.set_agents([random_agent, random_agent, random_agent, random_agent])

# env.set_agents([nfsp_agents[0], random_agent, random_agent, random_agent])
# env.set_agents([nfsp_agents[0], random_agent, nfsp_agents[1], random_agent])
# env.set_agents([nfsp_agents[0], nfsp_agents[1],nfsp_agents[2],nfsp_agents[3]])

# env.set_agents([nfsp_agents[0], rule_agent, rule_agent, rule_agent])
env.set_agents([nfsp_agents[0], rule_agent, nfsp_agents[1], rule_agent])

_trajectories, _payoffs = env.run(is_training=False)



In [37]:
from rlcard.games.tractor.utils import tournament_tractor
evaluate_num = 100
reward = tournament_tractor(env, evaluate_num)
print('Average win rate: ', reward)

# import numpy as np
# from rlcard.games.tractor.utils import CARD_RANK_STR

# for i in range(len(_trajectories[0])):
#     cards = []
#     for j in range(len(CARD_RANK_STR)):
#         if _trajectories[0][i][0]['obs'][0][1][j] == 1:
#             cards.append(CARD_RANK_STR[j])
#         if _trajectories[0][i][0]['obs'][0][2][j] == 1:
#             cards.append(CARD_RANK_STR[j])
#             cards.append(CARD_RANK_STR[j])
#     print("Round {}:".format(i))
#     print(','.join(cards))
#     print(env.game.round.trace[i*4:(i+1)*4])
#     print("Score: {}".format(_trajectories[0][i][2]))
#     print()

# print(env.game.round.score)


Average win rate:  [0.32, 0.65, 0.32, 0.65]
